In [ ]:
import datetime

In [ ]:
class Vert:
    def __init__(self):
        self.edges = dict()
        self.attrs = dict()
        
    def __repr__(self):
        return 'edges:' + str(self.edges) + ', attrs:' + str(self.attrs)

In [ ]:
minTime, maxTime = (1235000177, 1244362062)
timeRange = maxTime - minTime

In [4]:
import csv

G = dict()

with open('stackoverflow.csv') as so_file:
    reader = csv.reader(so_file)
    
    next(reader)
    for row in reader:
    
        # Asker ID
        i = row[2]
        # Answerer ID
        j = row[3]
        # Answer time
        at = int(row[11])
        # Tags as a list
        tags = row[5].split(',')
        
        if at < minTime:
            minTime = at
            
        if maxTime < at:
            maxTime = at
        
        ignoredUsers = ['NA', '0', '1']
        if i in ignoredUsers or j in ignoredUsers:
            continue
        
        if not i in G:
            G[i] = Vert()
    
        if not j in G:
            G[j] = Vert()
        
        if not j in G[i].edges or at > G[i].edges[j]:
            G[i].edges[j] = (at - minTime) / timeRange
        
        if not i in G[j].edges or at > G[j].edges[i]:
            G[j].edges[i] = (at - minTime) / timeRange
        
        for tag in tags:
            if not tag in G[i].attrs:
                G[i].attrs[tag] = 0
        
            if not tag in G[j].attrs:
                G[j].attrs[tag] = 0
            
            G[i].attrs[tag] += 1
            G[j].attrs[tag] += 1
    
print()
    
#usersRemoved = list(filter(lambda p: p[1] <= 3, sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True)))[2:]
usersRemoved = list(filter(lambda p: len(G[p].edges) < 3 or len(G[p].edges) > 300, G.keys()))

print(len(usersRemoved))
print(len(G))

inverseRemoved = list()
for rm in usersRemoved:
    for irm in G[rm].edges.keys():
        inverseRemoved.append( (irm, rm) )
    G.pop(rm)
    
for irm, rm in inverseRemoved:
    if irm in G:
        if rm in G[irm].edges:
            G[irm].edges.pop(rm)
            if len(G[irm].edges) <= 0:
                G.pop(irm)
    
print(len(G))
    
print()    

ranges = {
    (0, .1): 0,
    (.1, .2): 0,
    (.2, .3): 0,
    (.3, .4): 0,
    (.4, .5): 0,
    (.4, .5): 0,
    (.5, .6): 0,
    (.6, .7): 0,
    (.7, .8): 0,
    (.8, .9): 0,
    (.9, 1.1): 0
}
    
for k,v in G.items():
    for e in v.edges.values():
        for rs, re in ranges.keys():
            if rs <= e and e < re:
                ranges[(rs, re)] += 1
            


10357
11837
531



In [5]:
ranges

{(0, 0.1): 110,
 (0.1, 0.2): 124,
 (0.2, 0.3): 130,
 (0.3, 0.4): 144,
 (0.4, 0.5): 124,
 (0.5, 0.6): 152,
 (0.6, 0.7): 150,
 (0.7, 0.8): 142,
 (0.8, 0.9): 134,
 (0.9, 1.1): 158}

In [6]:
[]

[]

In [7]:
minTime, maxTime

(1235000177, 1244362062)

In [8]:
len(G.items()) # excluding '0' and '1' goes from 26883 to 11837

531

In [9]:
def commonNeighbors(cl, v):
    n_v = set(G[v].edges.keys())

    return len(cl.intersection(n_v)) / len(cl.union(n_v))

In [10]:
def getNextNode(cluster, clNeighbors):
    edgeWeightSums = [(e[0], sum(G[e[0]].edges[o] for o in e[1])) for e in [(n, set(cluster).intersection(set(G[n].edges.keys()))) for n in clNeighbors ]]
    sortedEdgeWeightSums = sorted(edgeWeightSums, key=lambda e: e[1], reverse=True)
    topEdgesByWeight = list(filter(lambda x: x[1] == sortedEdgeWeightSums[0][1], sortedEdgeWeightSums))
    return sorted([(t[0], t[1], len(set(G[t[0]].edges.keys()).difference(cluster).intersection(clNeighbors))) for t in topEdgesByWeight], key=lambda e: e[2], reverse=True)[0]

In [11]:
sorted([(u, len(G[u].edges)) for u in G.keys()], key=lambda u: u[1], reverse=True)

[('9', 115),
 ('10', 84),
 ('-3', 73),
 ('11', 63),
 ('12', 46),
 ('13', 45),
 ('-4', 45),
 ('14', 24),
 ('15', 23),
 ('-5', 19),
 ('16', 16),
 ('18', 14),
 ('19', 14),
 ('17', 12),
 ('51816', 11),
 ('20', 9),
 ('22', 9),
 ('23', 8),
 ('-6', 8),
 ('21', 7),
 ('24', 7),
 ('67396', 6),
 ('28', 6),
 ('-7', 6),
 ('33015', 6),
 ('26', 5),
 ('78259', 5),
 ('86751', 5),
 ('64878', 4),
 ('34537', 4),
 ('22088', 4),
 ('62970', 4),
 ('-8', 4),
 ('25', 4),
 ('53262', 4),
 ('356', 3),
 ('56555', 3),
 ('4639', 3),
 ('2635', 3),
 ('15124', 3),
 ('41021', 3),
 ('44', 3),
 ('63051', 3),
 ('39036', 3),
 ('26004', 3),
 ('32532', 3),
 ('27', 3),
 ('46914', 3),
 ('60711', 3),
 ('73137', 3),
 ('24545', 3),
 ('73628', 3),
 ('78627', 3),
 ('47775', 3),
 ('24702', 3),
 ('80359', 3),
 ('80112', 3),
 ('89818', 3),
 ('92735', 3),
 ('100051', 3),
 ('104015', 3),
 ('4653', 2),
 ('23072', 2),
 ('39532', 2),
 ('62391', 2),
 ('15054', 2),
 ('63213', 2),
 ('62195', 2),
 ('55102', 2),
 ('48553', 2),
 ('47645', 2),
 ('1

In [12]:
log = list()

def densityCluster(t_w):
    # 5 - 7
    nodeExists = dict()
    for node, edges_node in G.items():
        nodeExists[node] = True
    
    # 8
    clusters = list()
    
    # 9
    nodeDegree = dict()
    # 10 - 12
    for node, edges_node in G.items():
        nodeDegree[node] = len(edges_node.edges)
        
    
    for nId, __n_edges in sorted([(x[0], len(x[1].edges)) for x in G.items()],key=lambda y: y[1], reverse=True):
        node = str(nId)
        if nodeExists[node]:
            c = set()
            maxDegree = node
            nodeDegree[maxDegree] = -1
            nodeExists[maxDegree] = False
            neighbors = set(G[maxDegree].edges.keys())
            c.add(maxDegree)
            
            #for neighbor, edge_weight in neighbors:
                #print(float(edge_weight) + commonNeighbors(node, neighbor))
            
            #print(neighbors)
            
            while len(neighbors) > 0:
                neighbor, edge_weight, no_neighbors = getNextNode(c, neighbors)
                neighbors.remove(neighbor)
                
                if not nodeExists[neighbor]:
                    continue
                
                l = (edge_weight, no_neighbors)
                log.append(l)
                #print(l)
                
                if (edge_weight + commonNeighbors(c, neighbor)) >= t_w:
                    c.add(neighbor)
                    nodeExists[neighbor] = False
                    nodeDegree[neighbor] = -1
                    
                    newNeighbors = list(filter(lambda x: nodeExists[x], G[neighbor].edges.keys()))
                    neighbors.update( newNeighbors )
                                    
            #print(neighbors)
            
            clusters.append(c)
            
    return clusters


In [13]:
T_W = 1

print(datetime.datetime.now())
cs = sorted(densityCluster(T_W), key=lambda x: len(x), reverse=True)
cs = list(filter(lambda q: len(q) > 1, cs))
print(datetime.datetime.now())
[len(x) for x in cs]

2019-12-03 21:50:56.458108
2019-12-03 21:50:56.575382


[42, 16, 8, 6, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2]

In [16]:
for c in cs:
    topics = dict()
    
    for o in c:
        for t in G[o].attrs.keys():
            if not t in topics:
                topics[t] = 0
            topics[t] += 1
    
    print('Size: ' + str(len(c)))
    top5Topics = sorted(topics.items(), key=lambda t: t[1], reverse=True)[:5]
    for topi in top5Topics:
        print( str(round(topi[1] / len(c) * 100)) + '% ' +  topi[0] + ' (' + str(topi[1]) + ')')
    
    print()

    

Size: 42
69% best-practices (29)
67% c# (28)
64% ûnet (27)
64% subjective (27)
48% c++ (20)

Size: 16
50% best-practices (8)
38% java (6)
38% language-agnostic (6)
38% subjective (6)
38% c++ (6)

Size: 8
62% subjective (5)
50% c# (4)
50% c++ (4)
38% sqlserver (3)
38% beginner (3)

Size: 6
67% subjective (4)
67% programming-languages (4)
50% javascript (3)
50% java (3)
50% ûnet (3)

Size: 5
60% aspûnet (3)
60% beginner (3)
60% ûnet (3)
40% best-practices (2)
40% xml (2)

Size: 5
60% c# (3)
60% subjective (3)
60% javascript (3)
60% svn (3)
60% best-practices (3)

Size: 4
75% c# (3)
75% language-agnostic (3)
50% unit-testing (2)
50% ûnet (2)
50% job-interview (2)

Size: 4
100% java (4)
75% best-practices (3)
50% exceptions (2)
50% subjective (2)
50% fun (2)

Size: 4
75% best-practices (3)
50% sql (2)
50% sqlserver (2)
50% game-development (2)
50% resources (2)

Size: 3
100% subjective (3)
67% python (2)
67% python3û0 (2)
67% c (2)
67% c++ (2)

Size: 3
100% personal-development (3)
100% su

In [15]:
from pyvis.network import Network

coloredVerts = dict()

so_net = Network(height="100%", width="100%", bgcolor="#222222", font_color="white")
so_net.barnes_hut(spring_length=1000)

for src in G.keys():
    
    src_color = 'gray'
    if src in coloredVerts:
        src_color = coloredVerts[src]
    #else:
    #    continue
    
    so_net.add_node(src, label=" ", title=src, color=src_color)

    for dst, w in G[src].edges.items():
        dst_color = 'gray'
        if dst in coloredVerts:
            dst_color = coloredVerts[dst]
        #else:
        #    continue
        
        so_net.add_node(dst, label=" ", title=dst, color=src_color)

        
        so_net.add_edge(src, dst, value=w) #, physics=False)

for e in so_net.edges:
    e['title'] = e['value']
            
so_net.show_buttons(filter_=['nodes'])

so_net.save_graph("sover" + str(T_W) + ".html")

# Issues
 - High degree nodes cause clusters to consume all nodes
 - High degree nodes cause massive processing time